### **<모듈 임포트>**

In [ ]:
'''채우세요'''

#### **<GPU 확인, 사용할 device 설정>**

In [ ]:
'''채우세요'''

#### **<Seed/Random state 고정, 재현성을 위해>**

In [ ]:
'''채우세요'''

#### **<데이터 불러오기>**
#### 개미와 벌 분류하기
#### train data class당 대략 120개
#### validatation data class당 대략 70~80개

In [ ]:
data_transforms = {
    'train':transforms.Compose(['''채우세요''']),
    'val':transforms.Compose(['''채우세요''']),
}

In [ ]:
data_dir = '''채우세요'''
image_datasets = {x: datasets.'''채우세요'''(os.path.join(data_dir,x),data_transforms[x]) for x in ['train','val']}
dataloaders = {x: torch.utils.data.'''채우세요'''(image_datasets[x], batch_size=4, shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train','val']}
class_names = image_datasets['train'].classes

### **<pretrained VGG16 모델 불러오기>**

In [ ]:
vgg16 = models.'''채우세요'''

### **<fine-tuning : 마지막 Linear layer>**

In [ ]:
for param in vgg16.parameters():
    param.requires_grad = '''채우세요'''

In [ ]:
num_in = vgg16.classifier[-1].in_features
vgg16.classifier[-1] = nn.Linear(num_in,2)

In [ ]:
vgg16 = '''채우세요'''
criterion = '''채우세요'''
optimizer = optim.SGD('''채우세요''')
cus_lr_scheduler = lr_scheduler.StepLR('''채우세요''')

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                '''채우세요''' # Set model to training mode
            else:
                '''채우세요'''   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.'''채우세요'''
                
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = '''채우세요'''(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.'''채우세요'''
                        optimizer.'''채우세요'''

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0)) # channel, height, width -> height, width, channel
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1) # Given an interval, values outside the interval are clipped to the interval edges.
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

In [ ]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('real: {} | predicted: {}'.format(class_names[labels[j]],
                                                               class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

### **<모델 학습> :  train_model 함수 사용**

In [ ]:
vgg16_ft = train_model(vgg16, criterion, optimizer, 
                       cus_lr_scheduler,
                       num_epochs=25)

### **<모델 시각화> :  visualize_model 함수 사용**

In [ ]:
visualize_model(vgg16_ft)